In [219]:
import os
os.getcwd()

'C:\\Users\\sergey.astakhov\\Desktop\\BmstuDeepFake'

In [263]:
import pandas as pd
import numpy as np

In [264]:
df = pd.read_json("../deep_fake_src/dfdc_dataframes/dataframe_compact_total_27_frame_10_lim100_offset0.json")

face = df.filter(like='face', axis=1).copy()

for col in face:
    name = str(col) + '_reversed'
    df[name] = face[str(col)].map(lambda x: 1.0 / max(1,x))

df = df.filter(regex='^(.(?!(var)))*$', axis=1).filter(regex='^(.(?!(noise_1)))*$', axis=1)

df.head()

,index,filename,fake,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,...,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
0,0,aajrvbynqc.mp4,False,51984,11.603861,15.950859,22.036234,19.959687,13.405562,12.777617,...,30.0,23.0,9.0,18.0,0.392428,0.000019,0.015754,0.004667,1.0,1.0
1,1,aajrvbynqc.mp4,False,50176,15.218397,18.602677,20.166547,22.638240,15.031301,11.682786,...,9.0,10.0,8.0,20.0,0.408562,0.000020,0.015523,0.004635,1.0,1.0
2,2,aajrvbynqc.mp4,False,48400,17.841354,19.709579,22.040279,22.029932,15.643900,11.475191,...,4.0,7.0,6.0,18.0,0.425620,0.000021,0.015374,0.004661,1.0,1.0
3,3,aajrvbynqc.mp4,False,50176,14.979666,18.559879,21.796997,21.838523,15.232203,12.212530,...,3.0,11.0,9.0,14.0,0.412548,0.000020,0.015353,0.004538,1.0,1.0
4,4,aajrvbynqc.mp4,False,51076,14.595270,19.162088,21.454221,23.099652,15.149056,11.454065,...,5.0,11.0,7.0,22.0,0.403321,0.000020,0.015287,0.004425,1.0,1.0


In [265]:
df.shape

(1196, 281)

In [266]:
df_keys = pd.DataFrame(data=df[['filename', 'fake']], columns=["filename", "fake"])
df_keys_cnt = df_keys.groupby(["filename", "fake"])["filename"].count()
df_keys_cnt = pd.DataFrame(data=df_keys_cnt, columns=["filename", "cnt"])
df_keys_cnt["cnt"] = df_keys_cnt["filename"]
df_keys_cnt = df_keys_cnt.drop(columns=["filename"])
df_keys_cnt["_filename"] = df_keys_cnt.index
df_keys_cnt = df_keys_cnt.reset_index(drop=True)
df_keys_cnt["filename"] = df_keys_cnt["_filename"].map(lambda x: x[0])
df_keys_cnt["fake"] = df_keys_cnt["_filename"].map(lambda x: x[1])
# df_keys_cnt.columns = ["filename", "cnt"]
df_keys_cnt = df_keys_cnt.drop(columns=["_filename"])
df_keys_cnt.head()
# df_keys.head()

,cnt,filename,fake
0,11,aajrvbynqc.mp4,False
1,15,aasrzgbdpp.mp4,False
2,15,abplofmgwg.mp4,False
3,15,absytkidsf.mp4,True
4,15,abszlbwsaz.mp4,True


In [267]:
df_keys_cnt.shape

(89, 3)

In [268]:
df_keys_cnt = df_keys_cnt[df_keys_cnt["cnt"] > 8]
df_keys_cnt.shape

(81, 3)

In [269]:
df_keys_cnt = pd.concat([
    df_keys_cnt[df_keys_cnt.fake==True].sample(
        int(df_keys_cnt[df_keys_cnt.fake==False].shape[0]*1)), 
    df_keys_cnt[df_keys_cnt.fake==False]
])
df_keys_cnt.shape

(34, 3)

In [270]:
# from notebook2
# TODO: smart import through file

features = ['pt_24_std',
 'pt_57_hist_std',
 'pt_58_hist_noise_3',
 'pt_57_hist_noise_3',
 'pt_40_hist_noise_3',
 'pt_57_std',
 'pt_45_hist_noise_3',
 'pt_11_hist_noise_0.5',
 'pt_39_hist_noise_3',
 'pt_39_std',
 'pt_54_hist_noise_0.5',
 'overall_face_hist_noise_3',
 'pt_56_hist_noise_3',
 'pt_19_std',
 'pt_3_hist_noise_3',
 'overall_face_hist_noise_0.5',
 'overall_face_hist_std_reversed',
 'face_size_px_reversed',
 'overall_face_hist_noise_3_reversed',
 'pt_30_hist_noise_0.5',
 'overall_face_hist_noise_0.5_reversed',
 'pt_6_hist_noise_0.5',
 'overall_face_std_reversed',
 'pt_19_hist_noise_0.5',
 'pt_5_hist_noise_0.5',
 'overall_face_hist_std',
 'pt_24_hist_noise_0.5']
features

['pt_24_std',
 'pt_57_hist_std',
 'pt_58_hist_noise_3',
 'pt_57_hist_noise_3',
 'pt_40_hist_noise_3',
 'pt_57_std',
 'pt_45_hist_noise_3',
 'pt_11_hist_noise_0.5',
 'pt_39_hist_noise_3',
 'pt_39_std',
 'pt_54_hist_noise_0.5',
 'overall_face_hist_noise_3',
 'pt_56_hist_noise_3',
 'pt_19_std',
 'pt_3_hist_noise_3',
 'overall_face_hist_noise_0.5',
 'overall_face_hist_std_reversed',
 'face_size_px_reversed',
 'overall_face_hist_noise_3_reversed',
 'pt_30_hist_noise_0.5',
 'overall_face_hist_noise_0.5_reversed',
 'pt_6_hist_noise_0.5',
 'overall_face_std_reversed',
 'pt_19_hist_noise_0.5',
 'pt_5_hist_noise_0.5',
 'overall_face_hist_std',
 'pt_24_hist_noise_0.5']

In [271]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

keys_train, keys_test, _, _ = train_test_split( 
    df_keys_cnt, 
    df_keys_cnt,
    test_size=0.20, 
    random_state=420)
keys_train = pd.DataFrame(keys_train)
keys_test = pd.DataFrame(keys_test)
keys_train.head()

,cnt,filename,fake
24,15,agvtumzugg.mp4,False
60,15,aqnrvwmqbt.mp4,True
38,13,allirrddty.mp4,True
83,15,azeznwraac.mp4,True
0,11,aajrvbynqc.mp4,False


In [272]:
keys_test.head()

,cnt,filename,fake
44,14,amqvxdixtf.mp4,False
27,14,ahvyeeaweq.mp4,True
14,14,aeetrnfoir.mp4,False
1,15,aasrzgbdpp.mp4,False
69,15,avajqvytck.mp4,True


In [273]:
df_merged_train = keys_train.merge(df, on='filename')
df_merged_train["fake"] = df_merged_train["fake_x"]
df_merged_train = df_merged_train.drop(columns=["index", "fake_x", "fake_y"])

df_merged_test = keys_test.merge(df, on='filename')
df_merged_test["fake"] = df_merged_test["fake_x"]
df_merged_test = df_merged_test.drop(columns=["index", "fake_x", "fake_y"])

df_merged_train.head()

,cnt,filename,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,...,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed,fake
0,15,agvtumzugg.mp4,147456,10.893277,11.890595,16.386440,18.898326,19.519230,22.757824,18.976693,...,8.024691,17.592593,24.382716,0.149197,0.000007,0.016555,0.000972,1.0,1.0,False
1,15,agvtumzugg.mp4,142884,12.097904,14.136883,16.692377,22.783146,21.167736,24.854658,19.573899,...,7.098765,14.814815,25.000000,0.151871,0.000007,0.016483,0.000998,1.0,1.0,False
2,15,agvtumzugg.mp4,136900,7.140257,13.359327,17.753480,21.182231,17.750606,18.406671,19.126056,...,5.555556,15.740741,23.765432,0.153397,0.000007,0.016402,0.001058,1.0,1.0,False
3,15,agvtumzugg.mp4,141376,8.626907,11.802894,14.431912,19.344061,19.828922,23.511308,17.202038,...,7.716049,18.518519,26.234568,0.150662,0.000007,0.016414,0.000990,1.0,1.0,False
4,15,agvtumzugg.mp4,141376,6.031887,11.898907,18.087992,21.702544,24.425237,26.602919,18.223568,...,12.345679,20.679012,25.000000,0.151369,0.000007,0.016412,0.001021,1.0,1.0,False


In [274]:
df_merged_test.head()

,cnt,filename,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,...,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed,fake
0,14,amqvxdixtf.mp4,242064,7.424475,14.221669,13.125528,16.444131,17.816211,22.508383,15.946407,...,20.486111,3.993056,7.291667,0.105757,0.000004,0.013948,0.001008,1.0,1.0,False
1,14,amqvxdixtf.mp4,238144,10.034775,13.433478,12.733647,16.240580,15.806652,20.840714,15.253365,...,14.756944,4.687500,5.555556,0.107498,0.000004,0.013982,0.001071,1.0,1.0,False
2,14,amqvxdixtf.mp4,246016,11.607021,16.173592,15.081574,18.070015,17.796878,22.094241,14.308026,...,4.166667,5.034722,5.729167,0.104058,0.000004,0.014096,0.001215,1.0,1.0,False
3,14,amqvxdixtf.mp4,258064,14.524703,17.009207,15.435402,20.196257,18.327319,22.097712,14.105624,...,2.430556,3.993056,4.166667,0.098813,0.000004,0.013460,0.001226,1.0,1.0,False
4,14,amqvxdixtf.mp4,254016,16.204815,17.568219,17.405763,20.038220,16.188096,24.691349,15.470400,...,5.034722,7.465278,6.770833,0.100781,0.000004,0.013091,0.001128,1.0,1.0,False


In [275]:
df_merged_train.shape

(391, 281)

In [276]:
df_merged_test.shape

(102, 281)

In [277]:
X_train =  df_merged_train[features]
y_train = df_merged_train['fake']

X_test =  df_merged_test[features]
y_test = df_merged_test['fake']

pd.DataFrame(X_train).head()

,pt_24_std,pt_57_hist_std,pt_58_hist_noise_3,pt_57_hist_noise_3,pt_40_hist_noise_3,pt_57_std,pt_45_hist_noise_3,pt_11_hist_noise_0.5,pt_39_hist_noise_3,pt_39_std,...,face_size_px_reversed,overall_face_hist_noise_3_reversed,pt_30_hist_noise_0.5,overall_face_hist_noise_0.5_reversed,pt_6_hist_noise_0.5,overall_face_std_reversed,pt_19_hist_noise_0.5,pt_5_hist_noise_0.5,overall_face_hist_std,pt_24_hist_noise_0.5
0,21.421742,5.063633,11.111111,6.172840,21.913580,10.736914,20.679012,12.037037,20.370370,23.085527,...,0.000007,1.0,4.012346,1.0,0.617284,0.016555,0.617284,0.308642,1028.625171,2.160494
1,20.919811,4.111691,11.419753,9.259259,11.111111,15.599134,25.925926,7.716049,12.654321,13.905886,...,0.000007,1.0,3.395062,1.0,0.308642,0.016483,0.925926,0.000000,1001.694355,2.160494
2,21.914466,3.020721,16.049383,20.370370,16.049383,23.301710,24.691358,5.555556,13.271605,14.558556,...,0.000007,1.0,4.012346,1.0,0.000000,0.016402,0.000000,0.000000,945.345109,1.543210
3,20.912959,5.077500,10.185185,12.654321,11.419753,14.206607,25.000000,10.493827,11.111111,12.659377,...,0.000007,1.0,4.629630,1.0,0.000000,0.016414,1.234568,0.308642,1010.339756,2.469136
4,19.603606,6.664369,7.407407,7.716049,10.185185,9.823295,23.148148,7.716049,11.728395,12.353681,...,0.000007,1.0,2.469136,1.0,0.000000,0.016412,1.543210,0.000000,979.873465,1.234568


In [291]:
from joblib import dump, load
clf = load('random_forest_10.pkl')

In [292]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [293]:
# clf = clf.fit(X_train, y_train)

In [294]:
y_pred = clf.predict(X_test)
print(classification_report(
        y_test, 
        y_pred, 
        target_names=['class "real"', 'class "fakes"'], 
        zero_division=np.nan)
     )

               precision    recall  f1-score   support

 class "real"       0.66      0.63      0.64        73
class "fakes"       0.16      0.17      0.16        29

     accuracy                           0.50       102
    macro avg       0.41      0.40      0.40       102
 weighted avg       0.51      0.50      0.51       102



In [295]:
X_test

,pt_24_std,pt_57_hist_std,pt_58_hist_noise_3,pt_57_hist_noise_3,pt_40_hist_noise_3,pt_57_std,pt_45_hist_noise_3,pt_11_hist_noise_0.5,pt_39_hist_noise_3,pt_39_std,...,face_size_px_reversed,overall_face_hist_noise_3_reversed,pt_30_hist_noise_0.5,overall_face_hist_noise_0.5_reversed,pt_6_hist_noise_0.5,overall_face_std_reversed,pt_19_hist_noise_0.5,pt_5_hist_noise_0.5,overall_face_hist_std,pt_24_hist_noise_0.5
0,12.305270,7.324509,6.250000,5.208333,10.590278,9.758995,18.055556,0.000000,7.291667,11.556539,...,0.000004,1.0,5.902778,1.0,0.000000,0.013948,0.173611,0.173611,992.535337,0.520833
1,11.177256,7.541552,8.159722,6.423611,10.763889,11.873073,18.750000,0.000000,5.034722,9.958928,...,0.000004,1.0,4.861111,1.0,0.173611,0.013982,0.000000,0.173611,933.945369,0.520833
2,7.574549,7.296724,10.243056,7.812500,13.020833,15.752705,16.319444,0.173611,7.118056,13.135774,...,0.000004,1.0,4.166667,1.0,0.000000,0.014096,0.000000,6.423611,822.997318,0.520833
3,5.877052,5.741842,10.416667,9.375000,13.368056,19.231672,16.493056,0.347222,8.159722,15.521537,...,0.000004,1.0,4.861111,1.0,0.000000,0.013460,0.173611,6.770833,815.458223,0.694444
4,5.821899,5.217159,12.673611,11.979167,16.840278,22.800118,23.958333,0.347222,13.368056,21.475707,...,0.000004,1.0,4.861111,1.0,0.000000,0.013091,0.347222,7.118056,886.853101,0.347222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,12.873313,5.578606,8.264463,5.165289,4.545455,9.311278,2.272727,0.619835,5.371901,8.024953,...,0.000005,1.0,4.132231,1.0,2.479339,0.050797,0.619835,5.578512,2562.462779,0.619835
98,13.556679,5.794979,9.504132,5.371901,5.578512,10.717596,1.239669,0.826446,4.752066,7.536058,...,0.000005,1.0,4.132231,1.0,1.859504,0.049455,2.272727,3.719008,2368.412163,1.033058
99,6.044163,6.723599,9.504132,5.371901,5.785124,8.872114,1.652893,0.000000,4.752066,5.965415,...,0.000005,1.0,4.545455,1.0,2.892562,0.048183,1.446281,3.925620,2174.549424,0.826446
100,12.589558,7.392706,8.264463,5.371901,5.165289,8.025209,1.239669,1.446281,2.479339,4.229665,...,0.000005,1.0,2.479339,1.0,0.826446,0.052194,0.413223,1.652893,2476.152606,0.413223


In [296]:
pd.DataFrame(y_pred, columns=["prediction"])

,prediction
0,False
1,False
2,False
3,False
4,False
...,...
97,True
98,False
99,False
100,True


In [297]:
df_merged_test_cp = df_merged_test.copy()

In [298]:
df_merged_test_cp["prediction"] = y_pred
dictt = {True: 1, False: 0}
df_merged_test_cp["prediction_int"] = df_merged_test_cp["prediction"].map(lambda x: dictt[x])
df_merged_test_cp["prediction_match_int"] = (df_merged_test_cp["prediction"] == df_merged_test_cp["fake"])
df_merged_test_cp["prediction_match_int"] = df_merged_test_cp["prediction_match_int"].map(lambda x: dictt[x])

In [299]:
df_merged_test_cp

,cnt,filename,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,...,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed,fake,prediction,prediction_int,prediction_match_int
0,14,amqvxdixtf.mp4,242064,7.424475,14.221669,13.125528,16.444131,17.816211,22.508383,15.946407,...,0.105757,0.000004,0.013948,0.001008,1.0,1.0,False,False,0,1
1,14,amqvxdixtf.mp4,238144,10.034775,13.433478,12.733647,16.240580,15.806652,20.840714,15.253365,...,0.107498,0.000004,0.013982,0.001071,1.0,1.0,False,False,0,1
2,14,amqvxdixtf.mp4,246016,11.607021,16.173592,15.081574,18.070015,17.796878,22.094241,14.308026,...,0.104058,0.000004,0.014096,0.001215,1.0,1.0,False,False,0,1
3,14,amqvxdixtf.mp4,258064,14.524703,17.009207,15.435402,20.196257,18.327319,22.097712,14.105624,...,0.098813,0.000004,0.013460,0.001226,1.0,1.0,False,False,0,1
4,14,amqvxdixtf.mp4,254016,16.204815,17.568219,17.405763,20.038220,16.188096,24.691349,15.470400,...,0.100781,0.000004,0.013091,0.001128,1.0,1.0,False,False,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,15,aupwvhmmzg.mp4,206116,22.080527,21.699783,8.244713,8.328691,6.617381,8.378409,5.702705,...,0.049002,0.000005,0.050797,0.000390,1.0,1.0,False,True,1,0
98,15,aupwvhmmzg.mp4,202500,17.205647,15.201310,8.525174,8.316189,5.759232,4.548040,3.846814,...,0.049877,0.000005,0.049455,0.000422,1.0,1.0,False,False,0,1
99,15,aupwvhmmzg.mp4,200704,16.367508,20.777336,8.308080,12.957785,9.427804,8.168974,3.970793,...,0.050323,0.000005,0.048183,0.000460,1.0,1.0,False,False,0,1
100,15,aupwvhmmzg.mp4,207936,18.192666,24.936505,12.333167,15.870001,11.227830,8.428448,5.037809,...,0.047130,0.000005,0.052194,0.000404,1.0,1.0,False,True,1,0


In [300]:
# TODO: why division by zero in hists in notebook 1/2 ???